# Proyecto integrador

Miguel Ángel Pérez López A01750145

Ariadna Huesca Coronado A01749161

Lenguaje: GO

https://golang.org/ref/spec#Floating-point_literals

http://www.regular-expressions.info/unicode.html#prop

In [1]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

Variable

In [10]:
;;; Regular expression:
(def java-float #"(?xi)
                    [\p{L}_][\p{L}\d]*") 
(is (re-matches java-float "a"))
(is (re-matches java-float "_x9"))
(is (re-matches java-float "ThisVariableIsExported"))
(is (re-matches java-float "αβ"))
;(is (not (re-matches java-float ".")))

"αβ"

STRING

In [3]:
(def java-float #"(?xi)
                    \X") 

Syntax error reading source at (REPL:2:24).
Illegal/unsupported escape sequence near index 27
(?xi)
                    \X
                           ^


class clojure.lang.ExceptionInfo: 

In [38]:
;;; Regular expression:
(def java-float #"(?xi)
                    (`(\P{M}\p{M}*+)*`)
                    |(\"
                        [(\P{M}\p{M}*+)]*
                      \")") 
(is (re-matches java-float "`abc`"))
(is (re-matches java-float "`\n

               \n`"))
(is (re-matches java-float "\"\"\""))
(is (re-matches java-float "\"Hello, world!\n\""))
(is (re-matches java-float "\"日本語\""))
;(is (re-matches java-float "\"\u65e5本\U00008a9e\""))
;(is (re-matches java-float "\"\xff\u00FF\""))
;(is (not (re-matches java-float "\uD800")))
;(is (not (re-matches java-float "\U00110000")))

["\"日本語\"" nil nil "\"日本語\""]

In [3]:
(def regex #"(?xi)
    ( [+-]? \d+ [.] \d* (?: e [+-]? \d+)? )  # Grupo 1: Flotante
  | ( \d+ )                                  # Grupo 2: Entero
  | ( // .* )                                # Grupo 3: Comentario
  | ( [a-z] \w* )                            # Grupo 4: Variable
  | ( [=] )                                  # Grupo 5: Asignación
  | ( [+] )                                  # Grupo 6: Suma
  | ( [-] )                                  # Grupo 7: Resta
  | ( [*] )                                  # Grupo 8: Multiplicación
  | ( [/] )                                  # Grupo 9: División
  | ( \^ )                                   # Grupo 10: Potencia
  | ( [(] )                                  # Grupo 11: Paréntesis que abre
  | ( [)] )                                  # Grupo 12: Paréntesis que cierra
  | ( \s )                                   # Grupo 13: Espacios
  | ( . )                                    # Grupo 14: Carácter inválido
  ")

#'user/regex

In [4]:
(defn tokenize-file
  [file-name]
  (->> (re-seq regex (slurp file-name))
       (remove (fn [match] (match 13)))
       (map (fn [match]
              (let [token (match 0)]
                (cond
                  (match 1) [token "Flotante"]
                  (match 2) [token "Entero"]
                  (match 3) [token "Comentario"]
                  (match 4) [token "Variable"]
                  (match 5) [token "Asignación"]
                  (match 6) [token "Sum"]
                  (match 7) [token "Resta"]
                  (match 8) [token "Multiplicación"]
                  (match 9) [token "División"]
                  (match 10) [token "Potencia"]
                  (match 11) [token "Paréntesis que abre"]
                  (match 12) [token "Paréntesis que cierra"]
                  (match 14) [token "Carácter inválido"]))))))

#'user/tokenize-file

In [12]:
(tokenize-file "ejemplo.txt")

(["b" "Variable"] ["=" "Asignación"] ["7" "Entero"] ["a" "Variable"] ["=" "Asignación"] ["32.4" "Flotante"] ["*" "Multiplicación"] ["(" "Paréntesis que abre"] ["-8.6" "Flotante"] ["-" "Resta"] ["b" "Variable"] [")" "Paréntesis que cierra"] ["/" "División"] ["6.1E-8" "Flotante"] ["d" "Variable"] ["=" "Asignación"] ["a" "Variable"] ["^" "Potencia"] ["b" "Variable"] ["// Esto es un comentario" "Comentario"])

In [13]:
(defn table-lines
  [file-name]
  (->> (tokenize-file "ejemplo.txt")
      (map (fn [t] (format "%-30s%s" (t 0) (t 1))))))

#'user/table-lines

In [14]:
(defn line
  [character size]
  (clojure.string/join (repeat size character)))

#'user/line

In [15]:
(defn table-string 
  [file-name]
  (clojure.string/join \newline
    (concat [(line \= 50)]
            [(format "%-30s%s" "Token" "Tipo")]
            [(line \= 50)]
            (table-lines file-name)
            [(line \= 50)])))

#'user/table-string

In [16]:
(defn print-token-table
  [file-name]
  (println (table-string file-name)))


#'user/print-token-table

In [17]:
(print-token-table "ejemplo.txt")

Token                         Tipo
b                             Variable
=                             Asignación
7                             Entero
a                             Variable
=                             Asignación
32.4                          Flotante
*                             Multiplicación
(                             Paréntesis que abre
-8.6                          Flotante
-                             Resta
b                             Variable
)                             Paréntesis que cierra
/                             División
6.1E-8                        Flotante
d                             Variable
=                             Asignación
a                             Variable
^                             Potencia
b                             Variable
// Esto es un comentario      Comentario


nil

In [18]:
(spit "salida.txt" (table-string "ejemplo.txt"))

nil

In [50]:
(def mike #"(?xi)
 ( [+-]? \d+ [.] \d* (?: e [+-]? \d+)? ) #Flotante
| ( // .* ) #Comentario")

#'user/mike

In [53]:
(is (re-matches mike "4."))
(is (re-matches mike "//Hello"))

["//Hello" nil "//Hello"]